This notebook will demonstrate feature engineering and augmentation for the GRU/LSTM model.

GRU/LSTM part is mainly based on [OpenVaccine: Simple GRU Model](https://www.kaggle.com/xhlulu/openvaccine-simple-gru-model).


In [1]:
import pandas as pd
import numpy as np
import json
import tensorflow.keras.layers as L
import keras.backend as K
import tensorflow as tf
import plotly.express as px
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.cluster import KMeans
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
def allocate_gpu_memory(gpu_number=0):
    physical_devices = tf.config.experimental.list_physical_devices('GPU')

    if physical_devices:
        try:
            print("Found {} GPU(s)".format(len(physical_devices)))
            tf.config.set_visible_devices(physical_devices[gpu_number], 'GPU')
            tf.config.experimental.set_memory_growth(physical_devices[gpu_number], True)
            print("#{} GPU memory is allocated".format(gpu_number))
        except RuntimeError as e:
            print(e)
    else:
        print("Not enough GPU hardware devices available")
allocate_gpu_memory()

Ver='GRU_LSTM1'
aug_data = '../input/openvaccine-augmentation-data/aug_data1.csv'
debug = False

Found 1 GPU(s)
#0 GPU memory is allocated


In [2]:
def gru_layer(hidden_dim, dropout):
    return L.Bidirectional(L.GRU(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer = 'orthogonal'))

def lstm_layer(hidden_dim, dropout):
    return L.Bidirectional(L.LSTM(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer = 'orthogonal'))

def build_model(seq_len=107, pred_len=68, dropout=0.5, embed_dim=100, hidden_dim=256, type=0):
    inputs = L.Input(shape=(seq_len, 6))
    
    # split categorical and numerical features and concatenate them later.
    categorical_feat_dim = 3
    categorical_fea = inputs[:, :, :categorical_feat_dim]
    numerical_fea = inputs[:, :, 3:]

    embed = L.Embedding(input_dim=len(token2int), output_dim=embed_dim)(categorical_fea)
    reshaped = tf.reshape(embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3]))
    reshaped = L.concatenate([reshaped, numerical_fea], axis=2)
    
    if type == 0:
        hidden = gru_layer(hidden_dim, dropout)(reshaped)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
    elif type == 1:
        hidden = lstm_layer(hidden_dim, dropout)(reshaped)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
    elif type == 2:
        hidden = gru_layer(hidden_dim, dropout)(reshaped)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
    elif type == 3:
        hidden = lstm_layer(hidden_dim, dropout)(reshaped)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
    
    truncated = hidden[:, :pred_len]
    out = L.Dense(5, activation='linear')(truncated)
    model = tf.keras.Model(inputs=inputs, outputs=out)
    model.compile(tf.keras.optimizers.Adam(), loss=mcrmse)
    return model

In [3]:
token2int = {x:i for i, x in enumerate('().ACGUBEHIMSX')}
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

def preprocess_inputs(df, cols=['sequence', 'structure', 'predicted_loop_type']):
    base_fea = np.transpose(
        np.array(
            df[cols]
            .applymap(lambda seq: [token2int[x] for x in seq])
            .values
            .tolist()
        ),
        (0, 2, 1)
    )
    bpps_sum_fea = np.array(df['bpps_sum'].to_list())[:,:,np.newaxis]
    bpps_max_fea = np.array(df['bpps_max'].to_list())[:,:,np.newaxis]
    bpps_nb_fea = np.array(df['bpps_nb'].to_list())[:,:,np.newaxis]
    return np.concatenate([base_fea,bpps_sum_fea,bpps_max_fea,bpps_nb_fea], 2)

def rmse(y_actual, y_pred):
    mse = tf.keras.losses.mean_squared_error(y_actual, y_pred)
    return K.sqrt(mse)

def mcrmse(y_actual, y_pred, num_scored=len(pred_cols)):
    score = 0
    for i in range(num_scored):
        score += rmse(y_actual[:, :, i], y_pred[:, :, i]) / num_scored
    return score

## Load and preprocess data

In [4]:
train = pd.read_json('../input/stanford-covid-vaccine/train.json', lines=True)
test = pd.read_json('../input/stanford-covid-vaccine/test.json', lines=True)

In [5]:
# additional features

def read_bpps_sum(df):
    bpps_arr = []
    for mol_id in df.id.to_list():
        bpps_arr.append(np.load(f"../input/stanford-covid-vaccine/bpps/{mol_id}.npy").max(axis=1))
    return bpps_arr

def read_bpps_max(df):
    bpps_arr = []
    for mol_id in df.id.to_list():
        bpps_arr.append(np.load(f"../input/stanford-covid-vaccine/bpps/{mol_id}.npy").sum(axis=1))
    return bpps_arr

def read_bpps_nb(df):
    # normalized non-zero number
    # from https://www.kaggle.com/symyksr/openvaccine-deepergcn 
    bpps_nb_mean = 0.077522 # mean of bpps_nb across all training data
    bpps_nb_std = 0.08914   # std of bpps_nb across all training data
    bpps_arr = []
    for mol_id in df.id.to_list():
        bpps = np.load(f"../input/stanford-covid-vaccine/bpps/{mol_id}.npy")
        bpps_nb = (bpps > 0).sum(axis=0) / bpps.shape[0]
        bpps_nb = (bpps_nb - bpps_nb_mean) / bpps_nb_std
        bpps_arr.append(bpps_nb)
    return bpps_arr 

train['bpps_sum'] = read_bpps_sum(train)
test['bpps_sum'] = read_bpps_sum(test)
train['bpps_max'] = read_bpps_max(train)
test['bpps_max'] = read_bpps_max(test)
train['bpps_nb'] = read_bpps_nb(train)
test['bpps_nb'] = read_bpps_nb(test)

In [6]:
# clustering for GroupKFold
# expecting more accurate CV by putting similar RNAs into the same fold.
kmeans_model = KMeans(n_clusters=200, random_state=110).fit(preprocess_inputs(train)[:,:,0])
train['cluster_id'] = kmeans_model.labels_

## Data augmentation for training and TTA(test)

In [7]:
aug_df = pd.read_csv(aug_data)
display(aug_df.head())

,id,sequence,structure,log_gamma,score,cnt,predicted_loop_type
0,id_fff546103,GGAAAGCUAGGACGUGGGAGCGUAGCUCUCCACACGGGUACGCCAA...,.....((((((((((((((((...)))).)))).((((((((((.....,2,0.981885,3,EEEEESSSSSSSSSSSSSSSSHHHSSSSBSSSSMSSSSSSSSSSHH...
1,id_18ff9d670,GGAAAGAGCUCGUGAGAAGAAUCUAGUACAUGCAUACGCUACAUCU...,.....(((.((.......((..((((.....((....)).....))...,0,0.887485,5,EEEEESSSISSIIIIIIISSIISSSSIIIIISSHHHHSSIIIIISS...
2,id_177cd630b,GGAAAGAAGUAGCACGGUCCUAAGGUUACUGUAGCUAUGUCCAGCG...,(....((.((((((((((((...))..))))).))))).))(((.(...,2,0.923722,3,SMMMMSSISSSSSSSSSSSSHHHSSBBSSSSSBSSSSSISSSSSIS...
3,id_17a9ad5b7,GGAAAACACUGCAAAAGUCAACGAAGAAGUUGACUAAGAAGUGAUC...,......((((((...(((((((......))))))).....((((((...,2,0.977602,3,EEEEEESSSSSSMMMSSSSSSSHHHHHHSSSSSSSMMMMMSSSSSS...
4,id_17ab91518,GGAAAACGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCG...,......(((((((((((((((((((((((((((((....)))))))...,2,0.982851,3,EEEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSS...


This file was created using ARNIE, ViennaRNA and bpRNA in the following way.

Get candidate structures with different gamma values. 
See last cell of [How to Use ARNIE on Kaggle Notebook](https://www.kaggle.com/its7171/how-to-use-arnie-on-kaggle-notebook).

Remove the same as the original structure.

Get a structure with the largest score for each sequence.

Get the predicted_loop_type from the sequence and structure.
See [How to Generate predicted_loop_type](https://www.kaggle.com/its7171/how-to-generate-predicted-loop-type).

In [8]:
def aug_data(df):
    target_df = df.copy()
    new_df = aug_df[aug_df['id'].isin(target_df['id'])]
                         
    del target_df['structure']
    del target_df['predicted_loop_type']
    new_df = new_df.merge(target_df, on=['id','sequence'], how='left')

    df['cnt'] = df['id'].map(new_df[['id','cnt']].set_index('id').to_dict()['cnt'])
    df['log_gamma'] = 100
    df['score'] = 1.0
    df = df.append(new_df[df.columns])
    return df
train = aug_data(train)
test = aug_data(test)

In [9]:
if debug:
    train = train[:200]
    test = test[:200]

## Build and train model

In [10]:
model = build_model()
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 107, 6)]     0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 107, 3)]     0           input_1[0][0]                    
__________________________________________________________________________________________________
embedding (Embedding)           (None, 107, 3, 100)  1400        tf_op_layer_strided_slice[0][0]  
__________________________________________________________________________________________________
tf_op_layer_Reshape (TensorFlow [(None, 107, 300)]   0           embedding[0][0]                  
_______________________________________________________________________________________

In [11]:
def train_and_predict(type = 0, FOLD_N = 5):
    
    gkf = GroupKFold(n_splits=FOLD_N)

    public_df = test.query("seq_length == 107").copy()
    private_df = test.query("seq_length == 130").copy()

    public_inputs = preprocess_inputs(public_df)
    private_inputs = preprocess_inputs(private_df)


    holdouts = []
    holdout_preds = []

    for cv, (tr_idx, vl_idx) in enumerate(gkf.split(train,  train['reactivity'], train['cluster_id'])):
        trn = train.iloc[tr_idx]
        x_trn = preprocess_inputs(trn)
        y_trn = np.array(trn[pred_cols].values.tolist()).transpose((0, 2, 1))
        w_trn = np.log(trn.signal_to_noise+1.1)/2

        val = train.iloc[vl_idx]
        x_val_all = preprocess_inputs(val)
        val = val[val.SN_filter == 1]
        x_val = preprocess_inputs(val)
        y_val = np.array(val[pred_cols].values.tolist()).transpose((0, 2, 1))

        model = build_model(type=type)
        model_short = build_model(seq_len=107, pred_len=107,type=type)
        model_long = build_model(seq_len=130, pred_len=130,type=type)

        history = model.fit(
            x_trn, y_trn,
            validation_data = (x_val, y_val),
            batch_size=64,
            epochs=60,
            sample_weight=w_trn,
            callbacks=[
                tf.keras.callbacks.ReduceLROnPlateau(),
                tf.keras.callbacks.ModelCheckpoint(f'model{Ver}_cv{cv}.h5')
            ]
        )

        fig = px.line(
            history.history, y=['loss', 'val_loss'], 
            labels={'index': 'epoch', 'value': 'Mean Squared Error'}, 
            title='Training History')
        fig.show()

        model.load_weights(f'model{Ver}_cv{cv}.h5')
        model_short.load_weights(f'model{Ver}_cv{cv}.h5')
        model_long.load_weights(f'model{Ver}_cv{cv}.h5')

        holdouts.append(train.iloc[vl_idx])
        holdout_preds.append(model.predict(x_val_all))
        if cv == 0:
            public_preds = model_short.predict(public_inputs)/FOLD_N
            private_preds = model_long.predict(private_inputs)/FOLD_N
        else:
            public_preds += model_short.predict(public_inputs)/FOLD_N
            private_preds += model_long.predict(private_inputs)/FOLD_N
    return holdouts, holdout_preds, public_df, public_preds, private_df, private_preds

In [12]:
val_df, val_preds, test_df, test_preds = [], [], [], []
if debug:
    nmodel = 1
else:
    nmodel = 4
for i in range(nmodel):
    holdouts, holdout_preds, public_df, public_preds, private_df, private_preds = train_and_predict(i)
    val_df += holdouts
    val_preds += holdout_preds
    test_df.append(public_df)
    test_df.append(private_df)
    test_preds.append(public_preds)
    test_preds.append(private_preds)

Epoch 1/60
60/60 [==============================] - 5s 81ms/step - loss: 0.3357 - val_loss: 0.3567
Epoch 2/60
60/60 [==============================] - 3s 58ms/step - loss: 0.2804 - val_loss: 0.3146
Epoch 3/60
60/60 [==============================] - 4s 58ms/step - loss: 0.2623 - val_loss: 0.3007
Epoch 4/60
60/60 [==============================] - 4s 61ms/step - loss: 0.2519 - val_loss: 0.2953
Epoch 5/60
60/60 [==============================] - 3s 58ms/step - loss: 0.2406 - val_loss: 0.2706
Epoch 6/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2309 - val_loss: 0.2618
Epoch 7/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2240 - val_loss: 0.2547
Epoch 8/60
60/60 [==============================] - 4s 58ms/step - loss: 0.2178 - val_loss: 0.2503
Epoch 9/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2121 - val_loss: 0.2457
Epoch 10/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2062 - val_loss: 0.2417
Epoch 11/

Epoch 1/60
60/60 [==============================] - 5s 80ms/step - loss: 0.3327 - val_loss: 0.3448
Epoch 2/60
60/60 [==============================] - 4s 58ms/step - loss: 0.2780 - val_loss: 0.3112
Epoch 3/60
60/60 [==============================] - 4s 58ms/step - loss: 0.2600 - val_loss: 0.2955
Epoch 4/60
60/60 [==============================] - 4s 58ms/step - loss: 0.2480 - val_loss: 0.2830
Epoch 5/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2375 - val_loss: 0.2730
Epoch 6/60
60/60 [==============================] - 4s 58ms/step - loss: 0.2280 - val_loss: 0.2657
Epoch 7/60
60/60 [==============================] - 3s 58ms/step - loss: 0.2194 - val_loss: 0.2574
Epoch 8/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2127 - val_loss: 0.2527
Epoch 9/60
60/60 [==============================] - 4s 58ms/step - loss: 0.2067 - val_loss: 0.2481
Epoch 10/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2009 - val_loss: 0.2446
Epoch 11/

Epoch 1/60
60/60 [==============================] - 5s 78ms/step - loss: 0.3384 - val_loss: 0.3403
Epoch 2/60
60/60 [==============================] - 3s 58ms/step - loss: 0.2830 - val_loss: 0.3079
Epoch 3/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2636 - val_loss: 0.2921
Epoch 4/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2542 - val_loss: 0.2828
Epoch 5/60
60/60 [==============================] - 3s 58ms/step - loss: 0.2421 - val_loss: 0.2733
Epoch 6/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2313 - val_loss: 0.2584
Epoch 7/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2224 - val_loss: 0.2549
Epoch 8/60
60/60 [==============================] - 4s 58ms/step - loss: 0.2136 - val_loss: 0.2554
Epoch 9/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2077 - val_loss: 0.2451
Epoch 10/60
60/60 [==============================] - 3s 58ms/step - loss: 0.2024 - val_loss: 0.2470
Epoch 11/

Epoch 1/60
60/60 [==============================] - 5s 77ms/step - loss: 0.3288 - val_loss: 0.3531
Epoch 2/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2788 - val_loss: 0.3145
Epoch 3/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2594 - val_loss: 0.2938
Epoch 4/60
60/60 [==============================] - 3s 58ms/step - loss: 0.2499 - val_loss: 0.2823
Epoch 5/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2384 - val_loss: 0.2751
Epoch 6/60
60/60 [==============================] - 4s 58ms/step - loss: 0.2279 - val_loss: 0.2629
Epoch 7/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2194 - val_loss: 0.2554
Epoch 8/60
60/60 [==============================] - 4s 58ms/step - loss: 0.2138 - val_loss: 0.2486
Epoch 9/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2061 - val_loss: 0.2463
Epoch 10/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2014 - val_loss: 0.2423
Epoch 11/

Epoch 1/60
60/60 [==============================] - 5s 83ms/step - loss: 0.3315 - val_loss: 0.3436
Epoch 2/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2808 - val_loss: 0.2991
Epoch 3/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2611 - val_loss: 0.2871
Epoch 4/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2510 - val_loss: 0.2747
Epoch 5/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2387 - val_loss: 0.2655
Epoch 6/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2283 - val_loss: 0.2548
Epoch 7/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2193 - val_loss: 0.2452
Epoch 8/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2116 - val_loss: 0.2428
Epoch 9/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2052 - val_loss: 0.2372
Epoch 10/60
60/60 [==============================] - 4s 59ms/step - loss: 0.2007 - val_loss: 0.2354
Epoch 11/

...

Epoch 1/60
60/60 [==============================] - 5s 90ms/step - loss: 0.3423 - val_loss: 0.3668
Epoch 2/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2887 - val_loss: 0.3212
Epoch 3/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2636 - val_loss: 0.3080
Epoch 4/60
60/60 [==============================] - 4s 71ms/step - loss: 0.2529 - val_loss: 0.2916
Epoch 5/60
60/60 [==============================] - 4s 71ms/step - loss: 0.2441 - val_loss: 0.2793
Epoch 6/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2352 - val_loss: 0.2673
Epoch 7/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2264 - val_loss: 0.2609
Epoch 8/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2182 - val_loss: 0.2525
Epoch 9/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2112 - val_loss: 0.2501
Epoch 10/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2052 - val_loss: 0.2464
Epoch 11/

Epoch 1/60
60/60 [==============================] - 6s 93ms/step - loss: 0.3458 - val_loss: 0.3544
Epoch 2/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2863 - val_loss: 0.3133
Epoch 3/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2623 - val_loss: 0.2999
Epoch 4/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2520 - val_loss: 0.2903
Epoch 5/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2444 - val_loss: 0.2812
Epoch 6/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2349 - val_loss: 0.2756
Epoch 7/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2269 - val_loss: 0.2690
Epoch 8/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2190 - val_loss: 0.2587
Epoch 9/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2115 - val_loss: 0.2550
Epoch 10/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2048 - val_loss: 0.2518
Epoch 11/

Epoch 1/60
60/60 [==============================] - 5s 90ms/step - loss: 0.3416 - val_loss: 0.3460
Epoch 2/60
60/60 [==============================] - 4s 71ms/step - loss: 0.2871 - val_loss: 0.3048
Epoch 3/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2628 - val_loss: 0.2903
Epoch 4/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2512 - val_loss: 0.2837
Epoch 5/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2431 - val_loss: 0.2732
Epoch 6/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2329 - val_loss: 0.2694
Epoch 7/60
60/60 [==============================] - 4s 71ms/step - loss: 0.2222 - val_loss: 0.2603
Epoch 8/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2128 - val_loss: 0.2500
Epoch 9/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2051 - val_loss: 0.2468
Epoch 10/60
60/60 [==============================] - 4s 70ms/step - loss: 0.1993 - val_loss: 0.2476
Epoch 11/

Epoch 1/60
60/60 [==============================] - 5s 89ms/step - loss: 0.3407 - val_loss: 0.3632
Epoch 2/60
60/60 [==============================] - 4s 69ms/step - loss: 0.2851 - val_loss: 0.3134
Epoch 3/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2609 - val_loss: 0.2962
Epoch 4/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2504 - val_loss: 0.2866
Epoch 5/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2404 - val_loss: 0.2809
Epoch 6/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2318 - val_loss: 0.2700
Epoch 7/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2243 - val_loss: 0.2626
Epoch 8/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2137 - val_loss: 0.2533
Epoch 9/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2052 - val_loss: 0.2488
Epoch 10/60
60/60 [==============================] - 4s 70ms/step - loss: 0.1988 - val_loss: 0.2428
Epoch 11/

Epoch 1/60
60/60 [==============================] - 5s 89ms/step - loss: 0.3413 - val_loss: 0.3519
Epoch 2/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2868 - val_loss: 0.3017
Epoch 3/60
60/60 [==============================] - 4s 71ms/step - loss: 0.2631 - val_loss: 0.2876
Epoch 4/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2520 - val_loss: 0.2784
Epoch 5/60
60/60 [==============================] - 4s 71ms/step - loss: 0.2411 - val_loss: 0.2685
Epoch 6/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2311 - val_loss: 0.2614
Epoch 7/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2213 - val_loss: 0.2500
Epoch 8/60
60/60 [==============================] - 4s 70ms/step - loss: 0.2112 - val_loss: 0.2464
Epoch 9/60
60/60 [==============================] - 4s 71ms/step - loss: 0.2036 - val_loss: 0.2380
Epoch 10/60
60/60 [==============================] - 4s 72ms/step - loss: 0.1977 - val_loss: 0.2385
Epoch 11/

## post process

In [13]:
preds_ls = []
for df, preds in zip(test_df, test_preds):
    for i, uid in enumerate(df.id):
        single_pred = preds[i]
        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]
        preds_ls.append(single_df)
preds_df = pd.concat(preds_ls).groupby('id_seqpos').mean().reset_index()
# .mean() is for
# 1, Predictions from multiple models
# 2, TTA (augmented test data)

preds_ls = []
for df, preds in zip(val_df, val_preds):
    for i, uid in enumerate(df.id):
        single_pred = preds[i]
        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]
        single_df['SN_filter'] = df[df['id'] == uid].SN_filter.values[0]
        preds_ls.append(single_df)
holdouts_df = pd.concat(preds_ls).groupby('id_seqpos').mean().reset_index()

In [14]:
submission = preds_df[['id_seqpos', 'reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']]
submission.to_csv(f'submission.csv', index=False)
print(f'wrote to submission.csv')

wrote to submission.csv


## Validation

In [15]:
def print_mse(prd):
    val = pd.read_json('../input/stanford-covid-vaccine/train.json', lines=True)

    val_data = []
    for mol_id in val['id'].unique():
        sample_data = val.loc[val['id'] == mol_id]
        sample_seq_length = sample_data.seq_length.values[0]
        for i in range(68):
            sample_dict = {
                           'id_seqpos' : sample_data['id'].values[0] + '_' + str(i),
                           'reactivity_gt' : sample_data['reactivity'].values[0][i],
                           'deg_Mg_pH10_gt' : sample_data['deg_Mg_pH10'].values[0][i],
                           'deg_Mg_50C_gt' : sample_data['deg_Mg_50C'].values[0][i],
                           }
            val_data.append(sample_dict)
    val_data = pd.DataFrame(val_data)
    val_data = val_data.merge(prd, on='id_seqpos')

    rmses = []
    mses = []
    for col in ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']:
        rmse = ((val_data[col] - val_data[col+'_gt']) ** 2).mean() ** .5
        mse = ((val_data[col] - val_data[col+'_gt']) ** 2).mean()
        rmses.append(rmse)
        mses.append(mse)
        print(col, rmse, mse)
    print(np.mean(rmses), np.mean(mses))

In [16]:
print_mse(holdouts_df)

reactivity 0.6166591916253416 0.38026855861601966
deg_Mg_pH10 0.5563774587831174 0.30955587664195944
deg_Mg_50C 0.7501587826634389 0.5627381992070927
0.6410651443572992 0.4175208781550239


In [17]:
print_mse(holdouts_df[holdouts_df.SN_filter == 1])

reactivity 0.21930254046538877 0.04809360425457348
deg_Mg_pH10 0.26495890436670283 0.07020322100320359
deg_Mg_50C 0.22161146454909836 0.049111641219596276
0.23529096979373 0.055802822159124445
